In [2]:
import pandas as pd
from category_encoders import CatBoostEncoder

# ─── 1. 원본 데이터 불러오기 ──────────────────────────
train = pd.read_parquet("회원_신용_청구.parquet")
test = pd.read_parquet("회원_신용_청구_test.parquet")

# ─── 2. CatBoostEncoder를 이용해 Segment 숫자화 ───────
# Segment가 문자열일 때 → 숫자형으로 인코딩
encoder = CatBoostEncoder()
train['Segment_encoded'] = encoder.fit_transform(train[['Segment']], train['Segment'])

# ─── 3. 숫자형 열 선택 ──────────────────────────────
numeric_cols = train.select_dtypes(include='number').columns.tolist()

# ─── 4. Segment_encoded와의 상관계수 계산 ──────────────
correlations = train[numeric_cols].corr()['Segment_encoded'].drop('Segment_encoded')

# ─── 5. 절댓값 기준으로 상관계수 필터링 (예: 0.1 이상) ─
important_features = correlations[correlations.abs() > 0.1].index.tolist()

# ─── 6. 주요 변수 + ID + Segment로 구성된 훈련 데이터 저장 ─
columns_to_keep = ['ID', 'Segment'] + important_features
train_reduced = train[columns_to_keep]
train_reduced.to_csv("train_주요변수.csv", index=False)
print("✅ train_주요변수.csv 저장 완료")

# ─── 7. 테스트 파일에도 동일한 변수만 남기기 ─────────────
test_reduced = test[['ID'] + important_features]
test_reduced.to_csv("test_주요변수.csv", index=False)
print("✅ test_주요변수.csv 저장 완료")


✅ train_주요변수.csv 저장 완료
✅ test_주요변수.csv 저장 완료


In [4]:
correlations

기준년월              -0.000063
남녀구분코드             0.060331
연령                -0.004451
회원여부_이용가능_CA      -0.087540
소지카드수_이용가능_신용     -0.323115
입회일자_신용            0.252286
회원여부_연체           -0.027851
동의여부_한도증액안내       -0.049462
수신거부여부_메일         -0.106270
수신거부여부_SMS        -0.094514
가입통신회사코드          -0.074085
최종탈회후경과월           0.001828
탈회횟수_발급6개월이내       0.015517
거주시도명              0.007140
마케팅동의여부            0.047719
이용가능카드수_신용체크      -0.352251
이용카드수_신용_가족       -0.214798
이용금액_R3M_체크       -0.209966
_1순위카드이용건수        -0.383633
_1순위신용체크구분         0.012089
_2순위카드이용건수        -0.376548
최종카드발급일자           0.010832
이용여부_3M_해외겸용_본인   -0.231075
기본연회비_B0M          0.010118
제휴연회비_B0M         -0.019887
카드신청건수             0.038930
Life_Stage        -0.047110
최초한도금액            -0.107082
일시상환론한도금액         -0.094213
월상환론한도금액          -0.050292
CL이자율_할인전         -0.037258
RV약정청구율           -0.054579
RV최소결제비율          -0.031668
자발한도감액금액_R12M     -0.006141
자발한도감액후경과월         0.002106
강제한도감액금액_R12M      0